In [8]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

from collections import namedtuple
import pulp as plp
import logging
# import pyglpk

Item = namedtuple("Item", ['index', 'value', 'weight'])


def solve_it(input_data):
    logger = logging.getLogger('ds')
    logger.setLevel(logging.DEBUG)
    fh = logging.FileHandler('ds.log')
    fh.setLevel(logging.DEBUG)
    # create console handler with a higher log level
    ch = logging.StreamHandler()
    ch.setLevel(logging.ERROR)
    # create formatter and add it to the handlers
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    fh.setFormatter(formatter)
    ch.setFormatter(formatter)
    # add the handlers to the logger
    logger.addHandler(fh)
    logger.addHandler(ch)

    # Modify this code to run your optimization algorithm

    # just a comment line

    # parse the input
    lines = input_data.split('\n')

    firstLine = lines[0].split()
    item_count = int(firstLine[0])
    capacity = int(firstLine[1])-1
    set_I = range(0, item_count)

    items = []

    for i in range(1, item_count + 1):
        line = lines[i]
        parts = line.split()
        items.append(Item(i - 1, int(parts[0]), int(parts[1])))

    opt_model = plp.LpProblem(name="MIP Model")
    x_vars = {i:
                  plp.LpVariable(cat=plp.LpBinary, name="x_{0}".format(i))
              for i in set_I}

    constraint_capacity = plp.LpConstraint(
        e=plp.lpSum(items[i][2] * x_vars[i] for i in set_I),
        sense=plp.LpConstraintLE,
        rhs=capacity,
        name="constraint_Capacity")

    objective = plp.lpSum(x_vars[i] * items[i][1]
                          for i in set_I
                          )

    opt_model.addConstraint(constraint_capacity, "max capacity")
    opt_model.sense = plp.LpMaximize
    opt_model.setObjective(objective)

    opt_model.solve()
    # opt_model.solve(solver = GLPK_CMD())
    logger.info(plp.LpStatus[opt_model.status])
    # print(plp.value(opt_model.objective))
    taken = [0] * len(items)
    for variable in opt_model.variables():
        # logger.info("{} = {}".format(variable.name, variable.varValue))
        idx = int(str(variable.name).split('_')[1])
        taken[idx] = int(variable.varValue)

    weight = 0
    value = 0
    for i in range(0, item_count):
        weight += items[i][2] * taken[i]
        value += items[i][1] * taken[i]

    logger.info("Final weight: " + str(weight) + " capacity: " + str(capacity))
    logger.info("Final value: " + str(value))

    # # a trivial greedy algorithm for filling the knapsack
    # # it takes items in-order until the knapsack is full
    # value = 0
    # weight = 0
    # taken = [0]*len(items)
    #
    # for item in items:
    #     if weight + item.weight <= capacity:
    #         taken[item.index] = 1
    #         value += item.value
    #         weight += item.weight

    # prepare the solution in the specified output format
    # opt_df = pd.DataFrame.from_dict(x_vars, orient="index",
    #                             columns=["variable_object"])
    # opt_df["solution_value"] = opt_df["variable_object"].apply(lambda item: item.varValue)
    # taken = opt_df.drop(columns=["variable_object"], inplace=True)
    #
    # taken = opt_df.to_dict()
    output_data = str(int(plp.value(opt_model.objective))) + ' ' + str(0) + '\n'
    output_data += ' '.join(map(str, taken))
    return output_data

In [6]:
input_file = "data/knapsack/ks_4_0"

In [9]:
with open(input_file, 'r') as input_data_file:
    input_data = input_data_file.read()
print(solve_it(input_data))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/f13baa59b4de4a17a7698ee618fd2e0e-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/f13baa59b4de4a17a7698ee618fd2e0e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 6 COLUMNS
At line 23 RHS
At line 25 BOUNDS
At line 30 ENDATA
Problem MODEL has 1 rows, 4 columns and 4 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 19.875 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 1 rows, 4 columns (4 integer (4 of which binary)) and 4 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 1 integers unsatisfied sum - 0.1
Cbc0038I Solution found of -18
Cbc0038I Before mini b

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
